In [1]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from keras.utils import to_categorical 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding

Using TensorFlow backend.


# Model 1: One-Word-In, One-Word-Out Sequences 

"""
/Jack and Jill went up the hill To fetch 
a pail of water Jack fell down and broke 
his crown And Jill came tumbling after/

   X, y 
Jack, and 
and, Jill 
Jill, went
"""

In [2]:
# generate a sequence from the model 
def generate_seq(model, tokenizer, seed_text, n_words): 
    in_text, result = seed_text, seed_text 
    # generate a fixed number of words 
    for _ in range(n_words): 
        # encode the text as integer 
        encoded = tokenizer.texts_to_sequences([in_text])[0] 
        encoded = array(encoded) 
        # predict a word in the vocabulary 
        yhat = model.predict_classes(encoded, verbose=0) 
        # map predicted word index to word 
        out_word = '' 
        for word, index in tokenizer.word_index.items(): 
            if index == yhat: 
                out_word = word 
                break 
        # append to input 
        in_text, result = out_word, result + ' ' + out_word 
    return result


In [3]:
# define the model 
def define_model(vocab_size): 
    model = Sequential() 
    
    model.add(Embedding(vocab_size, 10, input_length=1)) 
    model.add(LSTM(50)) 
    model.add(Dense(vocab_size, activation='softmax'))
    # compile network 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # summarize defined model 
    model.summary() 
    plot_model(model, to_file='word_model.png', show_shapes=True) 
    return model


In [4]:
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n Jack fell down and broke his crown\n 
And Jill came tumbling after\n """

In [5]:
# integer encode text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
encoded = tokenizer.texts_to_sequences([data])[0] 

In [6]:
data.split(),len(data.split())

(['Jack',
  'and',
  'Jill',
  'went',
  'up',
  'the',
  'hill',
  'To',
  'fetch',
  'a',
  'pail',
  'of',
  'water',
  'Jack',
  'fell',
  'down',
  'and',
  'broke',
  'his',
  'crown',
  'And',
  'Jill',
  'came',
  'tumbling',
  'after'],
 25)

In [7]:
len(encoded),encoded

(25,
 [2,
  1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  1,
  16,
  17,
  18,
  1,
  3,
  19,
  20,
  21])

In [8]:
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 

Vocabulary Size: 22


In [9]:
# create word -> word sequences 
sequences = list() 
for i in range(1, len(encoded)): 
    sequence = encoded[i-1:i+1] 
    sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences)) 

Total Sequences: 24


In [10]:
sequences

[[2, 1],
 [1, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 2],
 [2, 14],
 [14, 15],
 [15, 1],
 [1, 16],
 [16, 17],
 [17, 18],
 [18, 1],
 [1, 3],
 [3, 19],
 [19, 20],
 [20, 21]]

In [11]:
# split into X and y elements 
sequences = array(sequences) 
X, y = sequences[:,0],sequences[:,1] 

In [12]:
X,y

(array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1,
        16, 17, 18,  1,  3, 19, 20]),
 array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1, 16,
        17, 18,  1,  3, 19, 20, 21]))

In [13]:
# one hot encode outputs 
y = to_categorical(y, num_classes=vocab_size) 

In [14]:
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [15]:
X.shape,y.shape

((24,), (24, 22))

In [16]:
# define model 
model = define_model(vocab_size)






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


In [17]:
# fit network 
model.fit(X, y, epochs=500, verbose=2) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/500
 - 2s - loss: 3.0910 - acc: 0.0417
Epoch 2/500
 - 0s - loss: 3.0902 - acc: 0.0833
Epoch 3/500
 - 0s - loss: 3.0895 - acc: 0.1667
Epoch 4/500
 - 0s - loss: 3.0888 - acc: 0.2083
Epoch 5/500
 - 0s - loss: 3.0880 - acc: 0.2083
Epoch 6/500
 - 0s - loss: 3.0873 - acc: 0.2083
Epoch 7/500
 - 0s - loss: 3.0865 - acc: 0.2083
Epoch 8/500
 - 0s - loss: 3.0857 - acc: 0.2083
Epoch 9/500
 - 0s - loss: 3.0849 - acc: 0.2083
Epoch 10/500
 - 0s - loss: 3.0841 - acc: 0.2083
Epoch 11/500
 - 0s - loss: 3.0832 - acc: 0.2083
Epoch 12/500
 - 0s - loss: 3.0824 - acc: 0.2083
Epoch 13/500
 - 0s - loss: 3.0815 - acc: 0.2083
Epoch 14/500
 - 0s - loss: 3.0807 - acc: 0.2083
Epoch 15/500
 - 0s - loss: 3.0798 - acc: 0.2083
Epoch 16/500
 - 0s - loss: 3.0789 - acc: 0.2083
Epoch 17/500
 - 0s - loss: 3.0779 - acc: 0.2083
Epoch 18/500
 - 0s - loss: 3.0770 - acc: 0.2083
Epoch 19/500
 - 0s - loss: 3.0760 - acc: 0.2083
Ep

Epoch 159/500
 - 0s - loss: 2.1447 - acc: 0.3333
Epoch 160/500
 - 0s - loss: 2.1348 - acc: 0.3333
Epoch 161/500
 - 0s - loss: 2.1248 - acc: 0.3333
Epoch 162/500
 - 0s - loss: 2.1148 - acc: 0.3750
Epoch 163/500
 - 0s - loss: 2.1049 - acc: 0.3750
Epoch 164/500
 - 0s - loss: 2.0949 - acc: 0.3750
Epoch 165/500
 - 0s - loss: 2.0849 - acc: 0.3750
Epoch 166/500
 - 0s - loss: 2.0749 - acc: 0.3750
Epoch 167/500
 - 0s - loss: 2.0649 - acc: 0.3750
Epoch 168/500
 - 0s - loss: 2.0549 - acc: 0.3750
Epoch 169/500
 - 0s - loss: 2.0448 - acc: 0.3750
Epoch 170/500
 - 0s - loss: 2.0347 - acc: 0.3750
Epoch 171/500
 - 0s - loss: 2.0247 - acc: 0.3750
Epoch 172/500
 - 0s - loss: 2.0145 - acc: 0.3750
Epoch 173/500
 - 0s - loss: 2.0044 - acc: 0.3750
Epoch 174/500
 - 0s - loss: 1.9942 - acc: 0.3750
Epoch 175/500
 - 0s - loss: 1.9840 - acc: 0.3750
Epoch 176/500
 - 0s - loss: 1.9738 - acc: 0.3750
Epoch 177/500
 - 0s - loss: 1.9635 - acc: 0.3750
Epoch 178/500
 - 0s - loss: 1.9532 - acc: 0.3750
Epoch 179/500
 - 0s 

Epoch 327/500
 - 0s - loss: 0.6462 - acc: 0.8333
Epoch 328/500
 - 0s - loss: 0.6411 - acc: 0.8333
Epoch 329/500
 - 0s - loss: 0.6360 - acc: 0.8333
Epoch 330/500
 - 0s - loss: 0.6310 - acc: 0.8333
Epoch 331/500
 - 0s - loss: 0.6261 - acc: 0.8333
Epoch 332/500
 - 0s - loss: 0.6212 - acc: 0.8333
Epoch 333/500
 - 0s - loss: 0.6163 - acc: 0.8333
Epoch 334/500
 - 0s - loss: 0.6115 - acc: 0.8333
Epoch 335/500
 - 0s - loss: 0.6068 - acc: 0.8333
Epoch 336/500
 - 0s - loss: 0.6021 - acc: 0.8333
Epoch 337/500
 - 0s - loss: 0.5974 - acc: 0.8333
Epoch 338/500
 - 0s - loss: 0.5927 - acc: 0.8333
Epoch 339/500
 - 0s - loss: 0.5882 - acc: 0.8333
Epoch 340/500
 - 0s - loss: 0.5837 - acc: 0.8333
Epoch 341/500
 - 0s - loss: 0.5792 - acc: 0.8333
Epoch 342/500
 - 0s - loss: 0.5748 - acc: 0.8333
Epoch 343/500
 - 0s - loss: 0.5704 - acc: 0.8333
Epoch 344/500
 - 0s - loss: 0.5660 - acc: 0.8333
Epoch 345/500
 - 0s - loss: 0.5617 - acc: 0.8333
Epoch 346/500
 - 0s - loss: 0.5575 - acc: 0.8333
Epoch 347/500
 - 0s 

Epoch 495/500
 - 0s - loss: 0.2602 - acc: 0.8750
Epoch 496/500
 - 0s - loss: 0.2596 - acc: 0.8750
Epoch 497/500
 - 0s - loss: 0.2590 - acc: 0.8750
Epoch 498/500
 - 0s - loss: 0.2584 - acc: 0.8750
Epoch 499/500
 - 0s - loss: 0.2578 - acc: 0.8750
Epoch 500/500
 - 0s - loss: 0.2572 - acc: 0.8750


In [19]:
# evaluate 
print(generate_seq(model, tokenizer, 'Jack', 6))


Jack and jill came tumbling after of
